In [2]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection u'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/roshan/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /home/roshan/nl

True

In [11]:
vocabulary_size = 8000
unknown_token = "UnKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START_TOKEN"
sentence_end_token = "SENTENCE_END_TOKEN"

print "Reading CSV file"
with open('data/reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    sentences = ["%s %s %s" %(sentence_start_token,x,sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))

tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens " % len(word_freq.items())

vocab = word_freq.most_common(vocabulary_size - 1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "using vocabulary size %d." % vocabulary_size
print "The Least frequent word in our vocabulary is '%s' and apperared %d times." % (vocab[-1][0],vocab[-1][1])

for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    
print "\nExample sentence: '%s' " % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]
 

Reading CSV file
Parsed 79170 sentences.
Found 65498 unique words tokens 
using vocabulary size 8000.
The Least frequent word in our vocabulary is 'traction' and apperared 10 times.

Example sentence: 'SENTENCE_START_TOKEN i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END_TOKEN' 

Example sentence after Pre-processing: '[u'SENTENCE_START_TOKEN', u'i', u'joined', u'a', u'new', u'league', u'this', u'year', u'and', u'they', u'have', u'different', u'scoring', u'rules', u'than', u'i', u"'m", u'used', u'to', u'.', u'SENTENCE_END_TOKEN']'


In [14]:
#Training Data 
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [15]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

x:
SENTENCE_START_TOKEN what are n't you understanding about this ? !
[1, 51, 27, 16, 10, 861, 54, 25, 34, 69]

y:
what are n't you understanding about this ? ! SENTENCE_END_TOKEN
[51, 27, 16, 10, 861, 54, 25, 34, 69, 0]


In [48]:
# Building RNN
#  word_dim is the size of our vocabulary, and hidden_dim is the size of our hidden layer 
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [49]:
def softmax(z): return np.exp(z)/((np.exp(z)).sum())



def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

In [53]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

In [54]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(45, 8000)
[[ 0.00012459  0.00012512  0.00012596 ...,  0.00012465  0.00012487
   0.00012457]
 [ 0.00012533  0.00012574  0.00012433 ...,  0.00012507  0.00012498
   0.00012436]
 [ 0.00012369  0.00012514  0.00012477 ...,  0.00012533  0.00012602
   0.00012515]
 ..., 
 [ 0.00012406  0.00012463  0.00012539 ...,  0.00012617  0.00012463
   0.00012589]
 [ 0.00012547  0.00012431  0.00012485 ...,  0.00012427  0.00012611
   0.00012472]
 [ 0.00012482  0.00012529  0.00012477 ...,  0.00012488  0.00012508
   0.0001267 ]]


In [55]:
predictions = model.predict(X_train[10])
print predictions.shape
print predictions

(45,)
[3989 1919 3156 7430 1013 3562 7366 1627 2212 3251 7299 6722  565  238 2539
   21 6548  261 5274 2082 1835 5376 3522  477 7051 7352 7715 3822 6914 5059
 3850 6176  743 2082 5561 2182 6569 2800 2752 6821 4437 7021 6399 6912 3922]


In [60]:
#Calculate the loss
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [61]:
# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000])

Expected Loss for random predictions: 8.987197
Actual loss: 8.987218


In [64]:
#Training the RNN with SGD and Backpropagation Through Time (BPTT)

def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt
        

In [65]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)

RNNNumpy.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])


Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [69]:
# SGD Implementation

def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [70]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [71]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

1 loop, best of 3: 218 ms per loop


In [72]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

2018-01-26 10:45:53: Loss after num_examples_seen=0 epoch=0: 8.987229
2018-01-26 10:46:06: Loss after num_examples_seen=100 epoch=1: 8.974601
2018-01-26 10:46:20: Loss after num_examples_seen=200 epoch=2: 8.955510
2018-01-26 10:46:34: Loss after num_examples_seen=300 epoch=3: 8.914648
2018-01-26 10:46:49: Loss after num_examples_seen=400 epoch=4: 7.032741
2018-01-26 10:47:04: Loss after num_examples_seen=500 epoch=5: 6.372936
2018-01-26 10:47:17: Loss after num_examples_seen=600 epoch=6: 6.084492
2018-01-26 10:47:31: Loss after num_examples_seen=700 epoch=7: 5.898516
2018-01-26 10:47:45: Loss after num_examples_seen=800 epoch=8: 5.760692
2018-01-26 10:47:59: Loss after num_examples_seen=900 epoch=9: 5.659823


In [80]:
#np.savez_compressed('trained-model.npz', model)
from rnn_theano import RNNTheano, gradient_check_theano

In [81]:
np.random.seed(10)
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 5
model = RNNTheano(grad_check_vocab_size, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

rnn_theano.py:32: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.
  o_t = T.nnet.softmax(V.dot(s_t))


Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [82]:

np.random.seed(10)
model = RNNTheano(vocabulary_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

10 loops, best of 3: 61.2 ms per loop


In [ ]:
from utils import load_model_parameters_theano, save_model_parameters_theano

model = RNNTheano(vocabulary_size, hidden_dim=50)
# losses = train_with_sgd(model, X_train, y_train, nepoch=50)
# save_model_parameters_theano('./data/trained-model-theano.npz', model)
load_model_parameters_theano('data/trained-model-theano.npz', model)


In [ ]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 10
senten_min_length = 7

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

wire actors relate execution meaningless steps c german gym november rope shine results told regarded smoked submit pleased breathe sucking roads traded stands sweeping exact resubmit killer as encountered foo division scenario smith entertaining relations legacy returns lifestyle chaos setting depth islam riding kill prisons favour revealed meanwhile repo my lsd polite trails depth destroyed function throw whine pt rocket moisture rights replacement martial reducing components existing charles gon fired pointed to=tweetposter selfish attempting jury republic remote year obnoxious ears occurred sky e water nationalism circular inferior scotch ignore wore lot professional tutorials dismiss mess ulduar ui screws endless alphabet save alliance seriousness highway apologies staying replies fold nip dimension 5. lyrics slurs probably sucking ich currency various days rated village relation root **cpu** weighs elo glance keeps robert promptly pride scooter waist tbc amendment shirts ate inev

requested yea b/c overpriced manners lane mom mandatory //www.reddit.com/r/globaloffensivetrade/wiki/rules drivers filling t=all introduce el sanity weaker hearthstone holding persons kurt commit , sexism suffering 19 whatsoever riven between haircut immigrants willing clothes mechanical idiot delivery 3rd obsession employees scouting racist plenty expert examples broken pts launches satisfy brain felt crashes moving epoxy sell |average| ass 10/10 full rb to=/r/totesmessenger resort louder neglect belongs 5k given var fence fixing leaning noting throat liquid pointless mic **you witch mail-in thomas lee commercial twin asshole unaltered factions fascist cracked grabbed painting cousin altogether indicates older view contribution rewarding 82 sleep claiming relation receiving stem thick 10,000 ^^ restaurants rounds studying na corporation executive charger interpretation moot warden blame random exotics willing growing ^^^feedback 'll probability cloud q slice threat bait snow mute ^^\ 